In [36]:
!pip install nltk pandas tensorflow-metal tensorflow-macos scikit-learn

  Using cached tensorflow_metal-1.2.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (1.3 kB)
  Using cached tensorflow_macos-2.16.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached tensorflow-2.16.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached protobuf-4.25.8-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
Using cached tensorflow_metal-1.2.0-cp311-cp311-macosx_12_0_arm64.whl (1.4 MB)
Using cached tensorflow_macos-2.16.2-cp311-cp311-macosx_12_0_arm64.whl (2.1 kB)
Using cached tensorflow-2.16.2-cp311-cp311-macosx_12_0_arm64.whl (227.0 MB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Us

In [71]:
import nltk
from nltk.util import ngrams
import pandas as pd

raw_text = ""
file_path = 'raw_data.txt' 
try:
    with open(file_path, 'r') as file:
        raw_text = file.read()
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")

clean_text = raw_text.replace('\n', ' ').lower()
words = clean_text.split()

window_size = 8
grams = list(ngrams(words, window_size))
print([g for g in grams if "isn't it" in " ".join(g)])  # Print the first 5 n-grams to verify
data = []

for gram in grams:
    # The first 6 words are our context (X)
    context = " ".join(gram[:-2])
    
    # The last 2 words are our target check
    target = f"{gram[-2]} {gram[-1]}"
    
    # Label 1 if the target is "isn't it", otherwise 0
    label = 1 if target == "isn't it" else 0
    
    data.append({"Context (X)": context, "Label (y)": label})

df = pd.DataFrame(data)

print("Positive Examples:")
print(df[df['Label (y)'] == 1].size)

print("\nNegative Examples (First 3):")
print(df[df['Label (y)'] == 0].size)

df.to_csv('processed_data.csv', index=False)

[('solving', 'df', 'by', 'dx', '=', '0', "isn't", 'it'), ('df', 'by', 'dx', '=', '0', "isn't", 'it', 'this'), ('by', 'dx', '=', '0', "isn't", 'it', 'this', 'is'), ('dx', '=', '0', "isn't", 'it', 'this', 'is', 'df'), ('=', '0', "isn't", 'it', 'this', 'is', 'df', 'by'), ('0', "isn't", 'it', 'this', 'is', 'df', 'by', 'dx'), ("isn't", 'it', 'this', 'is', 'df', 'by', 'dx', '='), ('is', 'not', 'straightforward', 'solving', 'is', 'difficult', "isn't", 'it'), ('not', 'straightforward', 'solving', 'is', 'difficult', "isn't", 'it', 'finding'), ('straightforward', 'solving', 'is', 'difficult', "isn't", 'it', 'finding', 'finding'), ('solving', 'is', 'difficult', "isn't", 'it', 'finding', 'finding', 'solution'), ('is', 'difficult', "isn't", 'it', 'finding', 'finding', 'solution', 'for'), ('difficult', "isn't", 'it', 'finding', 'finding', 'solution', 'for', 'this'), ("isn't", 'it', 'finding', 'finding', 'solution', 'for', 'this', 'this'), ('number', 'let', 'me', 'say', 'x', 'kn', "isn't", 'it'), ('l

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall


positives = df[df['Label (y)'] == 1]
negatives = df[df['Label (y)'] == 0]

# We are undersampling negatives to a 1:3 ratio. 
# 1138 positives * 3 = 3414 negatives. Total dataset will be 4552 rows.
negatives_sampled = negatives.sample(n=len(positives) * 3, random_state=42)

balanced_df = pd.concat([positives, negatives_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

X_text = balanced_df['Context (X)'].values
y = balanced_df['Label (y)'].values

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)

tokenizer = Tokenizer()
# Fit the tokenizer ONLY on the training data
tokenizer.fit_on_texts(X_train_text) 

# Convert the text into sequences of numbers
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

vocab_size = len(tokenizer.word_index) + 1 
max_length = 6

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='pre')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='pre')

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=[Precision(name='precision'), Recall(name='recall')]
)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print("Starting training...")
history = model.fit(
    X_train_padded, y_train, 
    validation_data=(X_test_padded, y_test), 
    epochs=30, 
    batch_size=32, # Standard batch size now that we aren't starving for data
    # callbacks=[early_stop],
    verbose=1
)

Starting training...
Epoch 1/30


/Users/parshipraneesh/Developer/ShakE/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6135 - precision: 0.2308 - recall: 0.0132 - val_loss: 0.5529 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5217 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.5318 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4567 - precision: 0.6667 - recall: 0.0396 - val_loss: 0.5582 - val_precision: 0.7000 - val_recall: 0.0614
Epoch 4/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4007 - precision: 0.7683 - recall: 0.2769 - val_loss: 0.5794 - val_precision: 0.4872 - val_recall: 0.1667
Epoch 5/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3585 - precision: 0.7645 - recall: 0.4637 - val_loss: 0.7033 - val_precision: 0.3519 - val_recall: 0.1667
Epoch 6/30
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3307 - precision: 0.7774 - recall: 0.5758 - val_loss: 0.7456 - val_precision: 0.3553 - val_recall

In [65]:
prediction_input = "multiple kernels here I have multiple kernels"
Xpred_seq = tokenizer.texts_to_sequences([prediction_input])
Xpred_padded = pad_sequences(Xpred_seq, maxlen=max_length, padding='pre')
probabilty = model.predict(Xpred_padded)
print(f"Predicted probability of 'isn't it': {probabilty[0][0]:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted probability of 'isn't it': 0.8045


In [69]:
#  save model
model.save('shakE_model.h5')